In [2]:
!pip install -q pandas chromadb sentence-transformers

In [3]:
%pip install -q pandas chromadb sentence-transformers


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3.13 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import openai
import json
import os
import time # (★★★ 이 줄이 추가되었습니다 ★★★)
import concurrent.futures

# --- 1단계: load_and_prepare_data 함수 (이전과 동일) ---
# ... (restaurant_summaries_output_100.csv 로드) ...
def load_and_prepare_data(csv_path):
  print(f"'{csv_path}' 파일 로드 중...")
  try:
    df = pd.read_csv(csv_path)
  except FileNotFoundError:
    print(f"[오류] 파일을 찾을 수 없습니다: {csv_path}")
    return None

  def safe_convert_to_dict(x):
    if pd.isna(x) or x == '{}' or x == '':
      return {}
    try:
      return ast.literal_eval(x)
    except Exception as e:
      print(f"메타데이터 파싱 오류: {e} \n데이터: {x}")
      return {"error": "parsing_failed"}

  print("메타데이터 컬럼을 딕셔너리로 변환 중...")
  df['메타데이터'] = df['메타데이터'].apply(safe_convert_to_dict)
  df['RAG텍스트'] = df['RAG텍스트'].fillna('')
  
  print(f"데이터 준비 완료: {len(df)}개")
  return df


# --- 2단계: build_vector_db 함수 (이전과 동일) ---
# ... (ChromaDB 구축) ...
def build_vector_db(df):
  print("\n--- 2단계: VectorDB 구축 시작 ---")
  
  # 1. 임베딩 모델 준비
  model_name = "distiluse-base-multilingual-cased-v1"
  sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=model_name
  )
  
  # 2. ChromaDB 클라이언트 초기화
  client = chromadb.PersistentClient(path="./restaurant_db")

  # 3. 컬렉션 생성
  collection_name = "restaurants"
  try:
    client.delete_collection(name=collection_name)
    print(f"기존 '{collection_name}' 컬렉션 삭제 완료.")
  except:
    pass
    
  collection = client.create_collection(
    name=collection_name,
    embedding_function=sentence_transformer_ef
  )

  # 4. 데이터 DB에 적재
  documents_list = df['RAG텍스트'].tolist()
  metadatas_list = df['메타데이터'].tolist() # (dict의 리스트)
  ids_list = df['id'].astype(str).tolist()

  print("ChromaDB 호환을 위해 metadata의 list 값을 string으로 변환 중...")
  processed_metadatas = []
  for metadata_dict in metadatas_list:
    processed_meta_item = {}
    for key, value in metadata_dict.items():
      if isinstance(value, list):
        processed_meta_item[key] = ",".join(map(str, value))
      else:
        processed_meta_item[key] = value
    processed_metadatas.append(processed_meta_item)

  print(f"ChromaDB에 데이터 {len(ids_list)}개 적재 중...")
  collection.add(
    documents=documents_list,
    metadatas=processed_metadatas,
    ids=ids_list
  )
  
  print("--- 2단계: VectorDB 구축 완료 ---")
  print(f"DB 경로: './restaurant_db'")
  print(f"총 {collection.count()}개의 레스토랑이 DB에 저장되었습니다.")
  return collection

# --- 3단계: 동적 하이브리드 검색 (★★★ 전체 수정됨 ★★★) ---

def build_filters_from_profile(user_filter_dict):
  """
  사용자 프로필 딕셔너리를 받아 ChromaDB 1차 필터와 Python 2차 필터로 분리합니다.
  """
  db_pre_filter_list = [] # 1차 필터 (ChromaDB가 처리)
  python_post_filter_dict = {} # 2차 필터 (Python이 처리)
  
  # ChromaDB가 처리할 수 있는 간단한 필터 키 (정확히 일치)
  # (이 키들은 $eq (같음) 연산만 가능합니다)
  DB_FILTER_KEYS = ['budget_range', 'spicy_available', 'vegetarian_options']
  
  # Python이 처리해야 하는 복잡한 필터 키 (포함 여부 검사)
  POST_FILTER_KEYS = ['main_ingredients_list', 'suitable_for']

  for key, value in user_filter_dict.items():
    if value == 'N/A': # 'N/A'는 필터링하지 않음
      continue
      
    if key in DB_FILTER_KEYS:
      # 1. ChromaDB 1차 필터 (간단한 $eq 매칭)
      # {'budget_range': '중'}
      db_pre_filter_list.append({key: value})
      
    elif key in POST_FILTER_KEYS:
      # 2. Python 2차 필터 (복잡한 '포함' 검사)
      # {'main_ingredients_list': "소고기,해산물,야채"}
      # (콤마로 구분된 문자열을 리스트로 변환)
      python_post_filter_dict[key] = value.split(',')
      
  # 1차 필터 리스트를 $and로 묶음
  db_pre_filter = {"$and": db_pre_filter_list} if db_pre_filter_list else {}
  
  return db_pre_filter, python_post_filter_dict

def run_hybrid_search(collection, user_profile_row, n_results=5):
  """
  사용자 프로필(Pandas Row)을 받아 동적 하이브리드 검색을 수행합니다.
  """
  print("\n--- 3단계: 하이브리드 검색 테스트 시작 ---")
  
  # 1. 사용자 프로필(Pandas Row)에서 데이터 추출
  try:
    user_rag_query = user_profile_row['rag_query_text']
    # 'filter_metadata_json' (문자열)을 딕셔너리로 변환
    user_filter_dict = ast.literal_eval(user_profile_row['filter_metadata_json'])
  except Exception as e:
    print(f"[오류] 사용자 프로필 파싱 실패: {e}")
    return

  # 2. (핵심) 사용자 프로필로 동적 필터 생성
  db_pre_filter, python_post_filter = build_filters_from_profile(user_filter_dict)
  
  REQUEST_N_RESULTS = 30 # 2차 필터링을 위해 넉넉하게 요청

  print(f"테스트 사용자: {user_profile_row['name']} ({user_profile_row['user_id']})")
  print(f"RAG 쿼리: '{user_rag_query[:100]}...'")
  print(f"DB 1차 필터 (ChromaDB): {db_pre_filter}")
  print(f"Python 2차 필터 (Post-Filter): {python_post_filter}")
  print(f"DB에 {REQUEST_N_RESULTS}개 요청 후, Python으로 2차 필터링 시작...")

  # 3. ChromaDB에 *1차 필터링* 및 *유사도 검색* 실행
  try:
    if db_pre_filter: # 필터가 있는 경우
      results = collection.query(
        query_texts=[user_rag_query],
        n_results=REQUEST_N_RESULTS,
        where=db_pre_filter
      )
    else: # 필터가 없는 경우 (예: 모든 조건이 'N/A'인 경우)
      results = collection.query(
        query_texts=[user_rag_query],
        n_results=REQUEST_N_RESULTS
      )
    
    print(f"\n--- 1차 검색 완료: {len(results['ids'][0])}개 후보 반환 ---")
    if not results.get('ids', [[]])[0]:
      print("1차 필터 조건에 맞는 식당을 찾지 못했습니다.")
      return

    # 4. Python으로 *2차 후-필터링* 실행
    final_results = []
    
    for i in range(len(results['ids'][0])):
      store_id = results['ids'][0][i]
      distance = results['distances'][0][i]
      rag_text = results['documents'][0][i]
      metadata = results['metadatas'][0][i]
      
      passes_post_filter = True
      
      # Python 2차 필터 딕셔너리({'main_ingredients_list': ['소고기', '해산물'], ...})를 순회
      for key, required_values_list in python_post_filter.items():
        
        # 가게의 메타데이터(문자열)를 가져옴 (예: "소고기,돼지고기")
        store_metadata_value_str = metadata.get(key, '')
        
        # (수정) 'OR'가 아닌 'AND' 로직으로 변경
        # (Soojin Kim은 '소고기' *그리고* '해산물' *그리고* '야채'를 선호)
        # (만약 'OR'를 원한다면 `all()`을 `any()`로 변경)
        if not all(req_val in store_metadata_value_str for req_val in required_values_list):
          passes_post_filter = False
          break # 이 가게는 2차 필터 탈락
      
      # 모든 2차 필터를 통과한 경우
      if passes_post_filter:
        final_results.append({
          "id": store_id,
          "distance": distance,
          "rag_text": rag_text,
          "metadata": metadata
        })
      
      if len(final_results) >= n_results:
        break 
    
    # --- 5. 최종 결과 출력 ---
    print("\n--- 3단계: 최종 하이브리드 검색 결과 (2차 필터링 완료) ---")
    if not final_results:
      print("조건에 맞는 식당을 찾지 못했습니다. (2차 필터링에서 모두 제외됨)")
      return

    # 결과 출력
    for i, item in enumerate(final_results):
      store_name = df[df['id'] == int(item['id'])]['가게'].values[0]
      
      print(f"\n[추천 순위 {i+1}] ID: {item['id']} (가게: {store_name})")
      print(f"  - 유사도(거리): {item['distance']:.4f}")
      print(f"  - (필터링된) RAG 텍스트: {item['rag_text'][:100]}...")
      print(f"  - (필터링된) 메타데이터: {item['metadata']}")
      
  except Exception as e:
    print(f"\n[오류] 쿼리 실행 중 오류 발생: {e}")


# --- 스크립트 실행 (★★★ 수정된 main ★★★) ---
if __name__ == "__main__":
  RESTAURANT_DB_FILE = "restaurant_summaries_output_100.csv"
  
  # (★★★ 수정됨 ★★★)
  # user_profiles_combined.csv 파일을 읽도록 경로 변경
  USER_PROFILE_FILE = "user_profiles_combined.csv" 

  # (★★★ 신규 ★★★)
  # 테스트할 사용자 프로필 수. (0 또는 500으로 설정 시 전체 실행)
  NUM_PROFILES_TO_TEST = 10
  
  # 1단계 (가게 DB 로드)
  df = load_and_prepare_data(RESTAURANT_DB_FILE) # (df는 가게 정보)

  if df is not None:
    # 2단계 (가게 VectorDB 구축)
    collection = build_vector_db(df)
    
    # 3단계 (사용자 프로필 로드 및 검색)
    if 'collection' in locals():
      try:
        # (★★★ 수정됨 ★★★)
        # 사용자 프로필 파일 전체를 먼저 로드
        user_df_all = pd.read_csv(USER_PROFILE_FILE)
        
        if user_df_all.empty:
          print(f"[오류] 사용자 프로필 파일({USER_PROFILE_FILE})이 비어있습니다.")
        else:
          
          # (★★★ 신규: 샘플링 로직 ★★★)
          if (
            NUM_PROFILES_TO_TEST > 0 and 
            NUM_PROFILES_TO_TEST < len(user_df_all)
          ):
            print(f"총 {len(user_df_all)}명의 사용자 프로필 중 {NUM_PROFILES_TO_TEST}개를 *랜덤 샘플링*하여 테스트합니다.")
            # 샘플링된 DataFrame
            user_df_to_run = user_df_all.sample(
              n=NUM_PROFILES_TO_TEST, 
              random_state=42 # 재현성을 위해 고정
            )
          else:
            print(f"총 {len(user_df_all)}명의 사용자 프로필 *전체*를 테스트합니다.")
            # 전체 DataFrame
            user_df_to_run = user_df_all
          # (★★★ 샘플링 로직 끝 ★★★)

          print(f"\n--- 총 {len(user_df_to_run)}명의 사용자 프로필로 하이브리드 검색을 시작합니다 ---")
          
          # (★★★ 수정됨 ★★★)
          # user_df_to_run (샘플링되거나 전체)을 순회하며 검색 실행
          # (enumerate를 사용하여 1, 2, 3... 카운터 'i'를 생성)
          for i, (original_index, user_profile_row) in enumerate(user_df_to_run.iterrows()):
            print("\n" + "#"*70)
            print(f"### [테스트 {i + 1}/{len(user_df_to_run)}] (Original Index: {original_index}) ###")
            
            run_hybrid_search(collection, user_profile_row)
            
            # (테스트 속도를 위해 1초 대기, 필요시 삭제)
            time.sleep(1) 
            
          print("\n" + "#"*70)
          print("--- 모든 사용자 프로필 검색이 완료되었습니다. ---")
          
      except FileNotFoundError:
        print(f"[오류] 사용자 프로필 파일을 찾지 못했습니다: {USER_PROFILE_FILE}")
      except Exception as e:
        print(f"[오류] 사용자 프로필 로드 또는 검색 중 오류 발생: {e}")

'restaurant_summaries_output_100.csv' 파일 로드 중...
메타데이터 컬럼을 딕셔너리로 변환 중...
데이터 준비 완료: 100개

--- 2단계: VectorDB 구축 시작 ---
기존 'restaurants' 컬렉션 삭제 완료.
ChromaDB 호환을 위해 metadata의 list 값을 string으로 변환 중...
ChromaDB에 데이터 100개 적재 중...
--- 2단계: VectorDB 구축 완료 ---
DB 경로: './restaurant_db'
총 100개의 레스토랑이 DB에 저장되었습니다.
총 500명의 사용자 프로필 중 10개를 *랜덤 샘플링*하여 테스트합니다.

--- 총 10명의 사용자 프로필로 하이브리드 검색을 시작합니다 ---

######################################################################
### [테스트 1/10] (Original Index: 361) ###

--- 3단계: 하이브리드 검색 테스트 시작 ---
[오류] 사용자 프로필 파싱 실패: 'rag_query_text'

######################################################################
### [테스트 2/10] (Original Index: 73) ###

--- 3단계: 하이브리드 검색 테스트 시작 ---
[오류] 사용자 프로필 파싱 실패: 'rag_query_text'

######################################################################
### [테스트 3/10] (Original Index: 374) ###

--- 3단계: 하이브리드 검색 테스트 시작 ---
[오류] 사용자 프로필 파싱 실패: 'rag_query_text'

######################################################################
##